## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/Weather_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Cayenne,GF,4.9333,-52.3333,79.09,1,75,5.99,broken clouds,2022-04-05 23:51:04
1,1,Mataura,NZ,-46.1927,168.8643,49.60,81,66,12.86,broken clouds,2022-04-05 23:51:04
2,2,La Ronge,CA,55.1001,-105.2842,46.65,39,20,11.50,few clouds,2022-04-05 23:51:04
3,3,Bethel,US,41.3712,-73.4140,50.40,57,98,1.01,overcast clouds,2022-04-05 23:51:05
4,4,Rikitea,PF,-23.1203,-134.9692,78.66,68,43,17.65,scattered clouds,2022-04-05 23:51:01


In [3]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Date                    object
dtype: object

In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Cayenne,GF,4.9333,-52.3333,79.09,1,75,5.99,broken clouds,2022-04-05 23:51:04
4,4,Rikitea,PF,-23.1203,-134.9692,78.66,68,43,17.65,scattered clouds,2022-04-05 23:51:01
6,6,Havelock,US,34.8791,-76.9013,70.02,82,75,9.22,broken clouds,2022-04-05 23:51:06
10,10,Bandarbeyla,SO,9.4942,50.8122,77.70,74,1,5.32,clear sky,2022-04-05 23:51:07
11,11,Tougue,GN,11.4667,-11.6000,79.83,21,70,2.73,broken clouds,2022-04-05 23:51:07
13,13,Monrovia,LR,6.3005,-10.7969,79.03,90,66,8.08,broken clouds,2022-04-05 23:48:15
15,15,Atuona,PF,-9.8000,-139.0333,79.34,71,1,22.44,clear sky,2022-04-05 23:51:08
16,16,Vila Velha,BR,-20.3297,-40.2925,82.35,82,0,9.22,clear sky,2022-04-05 23:51:08
21,21,Yulara,AU,-25.2406,130.9889,71.44,60,28,14.97,scattered clouds,2022-04-05 23:51:10
24,24,Kahului,US,20.8947,-156.4700,86.02,48,0,20.71,clear sky,2022-04-05 23:50:19


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                235
City                   235
Country                235
Lat                    235
Lng                    235
Max Temp               235
Humidity               235
Cloudiness             235
Wind Speed             235
Current Description    235
Date                   235
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df. count()

City_ID                235
City                   235
Country                235
Lat                    235
Lng                    235
Max Temp               235
Humidity               235
Cloudiness             235
Wind Speed             235
Current Description    235
Date                   235
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cayenne,GF,79.09,broken clouds,4.9333,-52.3333,
4,Rikitea,PF,78.66,scattered clouds,-23.1203,-134.9692,
6,Havelock,US,70.02,broken clouds,34.8791,-76.9013,
10,Bandarbeyla,SO,77.70,clear sky,9.4942,50.8122,
11,Tougue,GN,79.83,broken clouds,11.4667,-11.6000,
13,Monrovia,LR,79.03,broken clouds,6.3005,-10.7969,
15,Atuona,PF,79.34,clear sky,-9.8000,-139.0333,
16,Vila Velha,BR,82.35,clear sky,-20.3297,-40.2925,
21,Yulara,AU,71.44,scattered clouds,-25.2406,130.9889,
24,Kahului,US,86.02,clear sky,20.8947,-156.4700,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df= hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))